# NHITS Model

## Data Manipulation

In [10]:
from sklearn.metrics import mean_absolute_error
import pandas as pd

In [11]:
final_merged_df = pd.read_csv('../data/final_merged_data.csv', parse_dates=['Local Time'])

# Print preliminary data
print(f"Loaded {len(final_merged_df):,} records")
print(f"Number of unique rides: {final_merged_df['Ride'].nunique()}")

# Print a list of all unique ride names along with the number of records for each ride.
# Using value_counts() which is more reliable than manual counting
print(f"\nUnique rides:")
ride_counts = final_merged_df['Ride'].value_counts().sort_index()
for ride, count in ride_counts.items():
    print(f"  {ride}: {count:,} records")

/var/folders/pp/vwp6cpmd6xd962fpyb8gjv5w0000gn/T/ipykernel_76575/1020655066.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  final_merged_df = pd.read_csv('../data/final_merged_data.csv', parse_dates=['Local Time'])


Loaded 589,985 records
Number of unique rides: 42

Unique rides:
  Animation Academy: 18,021 records
  Games of Pixar Pier: 16,166 records
  Golden Zephyr: 18,021 records
  Goofy's Sky School: 18,021 records
  Grizzly River Run: 18,021 records
  Guardians of the Galaxy - Mission: BREAKOUT!: 18,021 records
  Guardians of the Galaxy - Monsters After Dark: 9,754 records
  Incredicoaster: 18,021 records
  Incredicoaster Single Rider: 18,021 records
  Inside Out Emotional Whirlwind: 18,021 records
  Jessie's Critter Carousel: 18,021 records
  Jumpin' Jellyfish: 18,021 records
  Luigi's Honkin' Haul-O-Ween: 1,516 records
  Luigi's Joy to the Whirl: 3,087 records
  Luigi's Rollickin' Roadsters: 11,564 records
  Mater's Graveyard JamBOOree: 3,344 records
  Mater's Jingle Jamboree: 3,087 records
  Mater's Junkyard Jamboree: 11,590 records
  Mickey's PhilharMagic: 18,021 records
  Monsters, Inc. Mike & Sulley to the Rescue!: 18,021 records
  Pixar Pal-A-Round - Swinging: 18,021 records
  Pixar P

In [12]:
# Dropping rides that are redundant/stupid
rides_to_drop = [
    'Animation Academy',
    'Games of Pixar Pier',
    'Guardians of the Galaxy - Monsters After Dark',
    'Luigi\'s Honkin\' Haul-O-Ween',
    'Luigi\'s Joy to the Whirl',
    'Mater\'s Graveyard JamBOOree',
    'Mater\'s Jingle Jamboree',
    'Red Car Trolley',
    'Redwood Creek Challenge Trail',
    'The Bakery Tour',
    'Sorcerer\'s Workshop',
    'Soarin\' Over California',
    'Rogers: The Musical', 
    'Walt Disney Imagineering Blue Sky Cellar', 
    'Turtle Talk with Crush', 
    'Mickey\'s PhilharMagic',
    'World of Color – ONE',
    'World of Color - Season of Light',
    'Villains Grove at Oogie Boogie Bash',
    'Silly Symphony Swings Single Rider',
    'Incredicoaster Single Rider',
    'Radiator Springs Racers Single Rider'
]

print(f"Rides before filtering: {final_merged_df['Ride'].nunique()}")
print(f"Total records before filtering: {len(final_merged_df):,}")

# Filter out the specified rides
final_merged_df = final_merged_df[~final_merged_df['Ride'].isin(rides_to_drop)]

print(f"Rides after filtering: {final_merged_df['Ride'].nunique()}")
print(f"Total records after filtering: {len(final_merged_df):,}")

print(f"\nDropped rides:")
for ride in rides_to_drop:
    print(f"  - {ride}")

Rides before filtering: 42
Total records before filtering: 589,985
Rides after filtering: 20
Total records after filtering: 344,631

Dropped rides:
  - Animation Academy
  - Games of Pixar Pier
  - Guardians of the Galaxy - Monsters After Dark
  - Luigi's Honkin' Haul-O-Ween
  - Luigi's Joy to the Whirl
  - Mater's Graveyard JamBOOree
  - Mater's Jingle Jamboree
  - Red Car Trolley
  - Redwood Creek Challenge Trail
  - The Bakery Tour
  - Sorcerer's Workshop
  - Soarin' Over California
  - Rogers: The Musical
  - Walt Disney Imagineering Blue Sky Cellar
  - Turtle Talk with Crush
  - Mickey's PhilharMagic
  - World of Color – ONE
  - World of Color - Season of Light
  - Villains Grove at Oogie Boogie Bash
  - Silly Symphony Swings Single Rider
  - Incredicoaster Single Rider
  - Radiator Springs Racers Single Rider


In [13]:
# Split the main dataset into separate, time-sorted DataFrames for each
# ride and store them in a dictionary for easy per-ride analysis.
ride_dataframes = {}

for ride in final_merged_df['Ride'].unique():
    ride_df = final_merged_df[final_merged_df['Ride'] == ride].copy()
    ride_df = ride_df.sort_values('Local Time').reset_index(drop=True)
    ride_dataframes[ride] = ride_df

for ride, df in ride_dataframes.items():
    
    # Feature Engineering (make raw data easier to use)
    df['hour'] = df['Local Time'].dt.hour
    df['dow'] = df['Local Time'].dt.dayofweek
    df['month'] = df['Local Time'].dt.month
    df['weekend'] = df['dow'].isin([5,6]).astype(int)

## NHITS

In [15]:
# Training NHITS

from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS

# Use the already-loaded DataFrame `final_merged_df`
df = ride_dataframes['Guardians of the Galaxy - Mission: BREAKOUT!'].copy()

# Rename to expected column names
df = df.rename(columns={
    'Local Time': 'ds',
    'Wait Time': 'y',
    'holiday_flag': 'holiday',
    'apparent_temperature (°F)': 'temp',
    'precipitation (mm)': 'prec'
})

df['unique_id'] = 'Guardians'

df = df[['unique_id', 'ds', 'y', 'tier', 'holiday', 'temp', 'prec', 'hour', 'dow', 'month', 'weekend']].reset_index(drop=True)
exog_cols = ['tier', 'holiday', 'temp', 'prec', 'hour', 'dow', 'month', 'weekend']

df['ds'] = pd.to_datetime(df['ds'])
df['ds'] = df['ds'].dt.floor('15min')

# Sort by ride and time
df = df.sort_values(['unique_id','ds'])

HOLDOUT_FRACTION = 0.3
split_idx = int(len(df) * (1 - HOLDOUT_FRACTION))
train = df.iloc[:split_idx].copy()
test = df.iloc[split_idx:].copy()

# Define NHITS model
model = NHITS(
    input_size=96,
    h=10,
    n_blocks=[2, 2, 2],                 # 3 stacks, each with 2 blocks
    mlp_units=[[256, 256],             # Stack 1 MLP
               [256, 256],             # Stack 2 MLP
               [256, 256]],            # Stack 3 MLP
    n_pool_kernel_size=[2, 4, 8],       # 3 stacks → 3 pool sizes
    batch_size=32,
    learning_rate=1e-3,
    max_steps=50000,
    hist_exog_list=exog_cols
)

# Wrap model
nf = NeuralForecast(models=[model], freq='15min')

# Fit
nf.fit(train)

Seed set to 1
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 1.7 M  | train
-------------------------------------------------------
1.7 M     Trainable params
0         Non-trainable params
1.7 M     Total params
6.919     Total estimated model params size (MB)
64        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=50000` reached.


In [16]:
fcst = nf.predict()
fcst.head()


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores


Predicting: |          | 0/? [00:00<?, ?it/s]

,unique_id,ds,NHITS
0,Guardians,2025-03-04 21:30:00+00:00,51.799747
1,Guardians,2025-03-04 21:45:00+00:00,50.611069
2,Guardians,2025-03-04 22:00:00+00:00,44.428596
3,Guardians,2025-03-04 22:15:00+00:00,53.803276
4,Guardians,2025-03-04 22:30:00+00:00,63.200657


In [17]:
# Get the test data we split earlier
print("Test data shape:", test.shape)
print("Predictions shape:", fcst.shape)

# Merge predictions with actual test values on timestamp
# The predictions should align with the first H=4 timesteps of the test set
merged_data = test.merge(fcst, on=['unique_id', 'ds'], how='inner')

print(f"Merged data shape: {merged_data.shape}")
print(f"Columns in merged data: {merged_data.columns.tolist()}")

if len(merged_data) > 0:
    # Calculate MAE
    actual_values = merged_data['y']
    predicted_values = merged_data['NHITS']
    
    mae = mean_absolute_error(actual_values, predicted_values)
    
    print(f"\nNHITS Model Results:")
    print(f"MAE: {mae:.2f} minutes")
    print(f"Number of predictions evaluated: {len(merged_data)}")
    
    # Show some sample predictions vs actual
    print(f"\nSample predictions vs actual:")
    comparison = merged_data[['ds', 'y', 'NHITS']].head(10)
    comparison['error'] = abs(comparison['y'] - comparison['NHITS'])
    print(comparison)
    
    # Summary statistics
    print(f"\nActual wait times - Mean: {actual_values.mean():.2f}, Std: {actual_values.std():.2f}")
    print(f"Predicted wait times - Mean: {predicted_values.mean():.2f}, Std: {predicted_values.std():.2f}")
    print(f"Average absolute error: {mae:.2f} minutes")
    print(f"MAPE: {100 * abs(actual_values - predicted_values).mean() / actual_values.mean():.2f}%")
    
else:
    print("No matching timestamps found between predictions and test data!")
    print("Test data time range:", test['ds'].min(), "to", test['ds'].max())
    print("Prediction time range:", fcst['ds'].min(), "to", fcst['ds'].max())

Test data shape: (5407, 11)
Predictions shape: (10, 3)
Merged data shape: (9, 12)
Columns in merged data: ['unique_id', 'ds', 'y', 'tier', 'holiday', 'temp', 'prec', 'hour', 'dow', 'month', 'weekend', 'NHITS']

NHITS Model Results:
MAE: 13.79 minutes
Number of predictions evaluated: 9

Sample predictions vs actual:
                         ds   y      NHITS      error
0 2025-03-04 21:45:00+00:00  50  50.611069   0.611069
1 2025-03-04 22:00:00+00:00  50  44.428596   5.571404
2 2025-03-04 22:15:00+00:00  50  53.803276   3.803276
3 2025-03-04 22:30:00+00:00  50  63.200657  13.200657
4 2025-03-04 22:45:00+00:00  50  49.724152   0.275848
5 2025-03-04 23:00:00+00:00  30  52.641045  22.641045
6 2025-03-04 23:15:00+00:00  55  56.379276   1.379276
7 2025-03-04 23:30:00+00:00  80  47.857754  32.142246
8 2025-03-04 23:45:00+00:00  90  45.491627  44.508373

Actual wait times - Mean: 56.11, Std: 17.99
Predicted wait times - Mean: 51.57, Std: 5.82
Average absolute error: 13.79 minutes
MAPE: 24.58%
